In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U spacy[cuda]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
Failed to build cupy
ERROR: Could not build wheels for cupy, which is required to install pyproject.toml-based projects


In [ ]:
import spacy
gpu = spacy.prefer_gpu()
print('GPU:', gpu)

GPU: False


In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 818.0 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import json

train_file = '/content/drive/MyDrive/project/train.json'
test_file = '/content/drive/MyDrive/project/test.json'

f = open(train_file, 'r')
train_data = json.loads(f.read())
f.close()

f = open(test_file, 'r')
test_data = json.loads(f.read())
f.close()

In [ ]:
#training_data = {'classes' : ['NAME_STUDENT', 'EMAIL', 'USERNAME', 'ID_NUM', 'PHONE_NUM', 'URL_PERSONAL', 'STREET_ADDRESS'], 'annotations' : []}
training_data = []

for essay in train_data:
  full_text = essay['full_text']
  tokens = essay['tokens']
  labels = essay['labels']

  label_indices = []
  label_type = []
  PII_type = []
  for i in range(len(labels)):
    if labels[i] != 'O':
      label_indices.append(i)
      if labels[i][0] == 'B':
        label_type.append('B')
        PII_type.append(labels[i][2:])
      else:
        label_type.append('I')

  essay_indices = []
  txt_index = 0
  for i in label_indices:
    txt_index = full_text.find(tokens[i], txt_index)
    essay_indices.append(txt_index)
    txt_index = txt_index + len(tokens[i]) - 1

  start_end_PII = []
  PII_count = 0
  for i in range(len(label_indices)):
    if label_type[i] == 'B':
      start_index = essay_indices[i]
      end_index = start_index + len(tokens[label_indices[i]])
      check_len = i + 1

      while check_len < len(label_indices) and label_type[check_len] == 'I':
        end_index = essay_indices[check_len] + len(tokens[label_indices[check_len]])
        check_len += 1

      start_end_PII.append((start_index, end_index, PII_type[PII_count]))
      PII_count += 1

  #temp_dict = {}
  #temp_dict['entities'] = start_end_PII
  #temp_dict['text'] = full_text
  #training_data['annotations'].append(temp_dict)
  training_data.append((full_text, start_end_PII))

In [ ]:
test_essays = []
for essay in test_data:
  test_essays.append(essay['full_text'])

In [ ]:
import random

random.shuffle(training_data)
training_data_subset = training_data[0:500]

#train_split = int(len(training_data_subset) * 0.8)

#train_data = training_data_subset[:train_split]
#dev_data = training_data_subset[train_split:]

In [ ]:
print(len(training_data_subset))

500


In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

def convert(path, dataset):
  spacy.require_gpu()
  nlp = spacy.blank("en")
  spacy.require_gpu()
  db = DocBin()

  for text, annotations in dataset:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
      span = doc.char_span(start, end, label=label, alignment_mode="contract")
      ents.append(span)
    doc.ents = ents
    db.add(doc)

  db.to_disk(path)

In [ ]:
convert("train.spacy", training_data_subset)
#convert("train.spacy", train_data)
#convert("dev.spacy", dev_data)

In [ ]:
 !python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy debug data /content/drive/MyDrive/project/config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
5445 training docs
1362 evaluation docs
✔ No overlap between training and evaluation data

============================== Vocab & Vectors ==============================
ℹ 3995294 total word(s) in the data (48345 unique)
ℹ 514157 vectors (514157 unique keys, 300 dimensions)
⚠ 324853 words in training data without vectors (8%)

========================== Named Entity Recognition ==========================
ℹ 7 label(s)
0 missing value(s) (tokens with '-' label)
⚠ Low number of examples for label 'STREET_ADDRESS' (2)
⚠ Low number of examples for label 'USERNAME' (6)
⚠ Low number of examples for label 'EMAIL' (32)
⚠ Low number of examples for label 'PHONE_NUM' (6)
✔ Examples without occurrences available for all labels
✔ No entit

In [ ]:
!python -m spacy train /content/drive/MyDrive/project/config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    246.05    0.00    0.00    0.00    0.00
  0     200        473.78   3464.56   36.99   69.57   25.20    0.37
  0     400         15.40    102.91   23.13   85.00   13.39    0.23
  1     600         17.33     56.97   62.39   68.22   57.48    0.62
  1     800         17.18     48.55   48.15   39.59   61.42    0.48
  2    1000         53.38    104.24   25.95   41.38   18.90    0.26
  2    1200         26.84     64.97   54.55   85.00   40.16    0.55
  2    1400         19.56     71.91   61.68   75.86   5

In [ ]:
!zip -r /content/drive/MyDrive/project/model.zip model-best

  adding: model-best/ (stored 0%)
  adding: model-best/vocab/ (stored 0%)
  adding: model-best/vocab/key2row (deflated 16%)
  adding: model-best/vocab/lookups.bin (stored 0%)
  adding: model-best/vocab/strings.json (deflated 77%)
  adding: model-best/vocab/vectors.cfg (stored 0%)
  adding: model-best/vocab/vectors (deflated 8%)
  adding: model-best/tokenizer (deflated 81%)
  adding: model-best/meta.json (deflated 62%)
  adding: model-best/ner/ (stored 0%)
  adding: model-best/ner/cfg (deflated 33%)
  adding: model-best/ner/moves (deflated 65%)
  adding: model-best/ner/model (deflated 9%)
  adding: model-best/config.cfg (deflated 60%)
  adding: model-best/tok2vec/ (stored 0%)
  adding: model-best/tok2vec/cfg (stored 0%)
  adding: model-best/tok2vec/model (deflated 8%)


In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/project/model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm
nlp1 = spacy.load(r"/content/drive/MyDrive/project/model-best")

In [ ]:
total = 0
correct = 0
retrieved = 0

for text, annotations in tqdm(training_data):
  if len(annotations) != 0:
    model_annotations = []
    doc = nlp1(text)

    for ent in doc.ents:
      model_annotations.append((ent.start_char, ent.end_char, ent.label_))

    for model_annotation in model_annotations:
      if model_annotation in annotations:
        correct += 1

    retrieved += len(model_annotations)

    total += len(annotations)

100%|██████████| 6807/6807 [04:15<00:00, 26.65it/s]  


In [ ]:
print(correct)
print(retrieved)
print(total)

906
1059
1606


In [ ]:
print("precision: %f" % (correct / retrieved))
print("Accuracy: %f" % (correct / total))

precision: 0.855524
Accuracy: 0.564134


In [ ]:
print("Accuracy: %f" % (correct / total * 100))

Accuracy: 56.413450


In [ ]:
#doc = nlp1(test_essays[0])

In [ ]:
for ent in doc.ents:
  print(ent.start_char)
  print(ent.end_char)
  print(ent.label_)

In [ ]:
colors = {"NAME_STUDENT": "#F67DE3", "EMAIL": "#7DF6D9", "USERNAME":"#FFFFFF", "ID_NUM":"#9D876C", "PHONE_NUMBER":"#32ADD5", "URL_PERSONAL":"#631881", "STREET_ADDRESS":"#E1ACE2"}
options = {"colors": colors}

In [ ]:
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)